<a href="https://colab.research.google.com/github/VecToR-1024/urban-map/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import pandas as pd
import numpy as np
from sklearn.neighbors import RadiusNeighborsRegressor
import matplotlib.pyplot as plt

# Model for Government Priority Index (GPI)

In [23]:
# 1. Load Data
df = pd.read_csv('GovPriorityIndex.csv')


# Remove rows where Coordinates OR Score are missing (NaN)
df = df.dropna(subset=['geometry/coordinates/1', 'geometry/coordinates/0', df.columns[0]])

# 2. Prepare Coordinates
target_col = df.columns[0]
X = np.radians(df[['geometry/coordinates/1', 'geometry/coordinates/0']].values)
y = df[target_col].values

# 3. Model
radius_miles = 0.8
radius_radians = radius_miles / 3959.0

model = RadiusNeighborsRegressor(
    radius=radius_radians,
    weights='distance',
    metric='haversine',
    algorithm='ball_tree'
)

model.fit(X, y)


RadiusNeighborsRegressor(algorithm='ball_tree', metric='haversine',
                         radius=0.0002020712301086133, weights='distance')

In [24]:
def get_score(lat, lon):
    """
    Predicts score for a given lat/lon based on neighbors within 0.8 miles.
    """
    query_point = np.radians([[lat, lon]])

    try:
        score = model.predict(query_point)[0]
        return round(score, 4)
    except Exception:
        return None

# Example Usage: Prediction for a point in Seattle
print(f"Predicted Score: {get_score(47.61053462,-122.3408738)}")

Predicted Score: 0.9697


In [27]:
# 1. Define Grid Bounds & Resolution
lat_min, lat_max = df['geometry/coordinates/1'].min(), df['geometry/coordinates/1'].max()
lon_min, lon_max = df['geometry/coordinates/0'].min(), df['geometry/coordinates/0'].max()

# Resolution: Adjust '500' to change how "blocky" the grid is
# 500 means the image will be 500 squares wide
width_pixels = 100
aspect_ratio = (lat_max - lat_min) / (lon_max - lon_min)
height_pixels = int(width_pixels * aspect_ratio) # Keep map proportions correct

print(f"Generating grid: {width_pixels} x {height_pixels}")

# 2. Generate Grid Centers
lat_linspace = np.linspace(lat_min, lat_max, height_pixels)
lon_linspace = np.linspace(lon_min, lon_max, width_pixels)
grid_x_lon, grid_y_lat = np.meshgrid(lon_linspace, lat_linspace)

# Flatten to list of (Lat, Lon) points for batch prediction
# Note: grid_y_lat is Lat, grid_x_lon is Lon
query_points = np.column_stack((grid_y_lat.ravel(), grid_x_lon.ravel()))
query_points_rad = np.radians(query_points)

# 3. Batch Prediction with Zero-Filling
# We predict in chunks if memory is an issue, but for <1M points, one go is fine.
try:
    # RadiusNeighborsRegressor returns 0.0 by default for points with no neighbors
    # if you set it up correctly, or raises an error.
    # To be safe for "Transparency", we create a mask.

    # Get neighbors count first to identify "empty" cells (transparency mask)
    neighbor_graph = model.radius_neighbors(query_points_rad, return_distance=False)
    has_neighbors = np.array([len(n) > 0 for n in neighbor_graph])

    # Predict scores
    raw_scores = model.predict(query_points_rad)

    # Apply transparency logic: Set score to NaN where no neighbors exist
    # (Matplotlib treats NaN as transparent automatically)
    final_scores = np.where(has_neighbors, raw_scores, np.nan)

except Exception as e:
    print(f"Prediction logic warning: {e}")
    # Fallback: just predict and assume 0s
    final_scores = model.predict(query_points_rad)
    final_scores[final_scores == 0] = np.nan # Assume exactly 0.0 means "no data"

# 4. Reshape for Image
# Reshape to (Height, Width)
grid_matrix = final_scores.reshape(height_pixels, width_pixels)

# Flip Y-axis (Images start from top-left, Maps start from bottom-left)
grid_matrix = np.flipud(grid_matrix)

# 5. Save as PNG
plt.figure(figsize=(12, 12 * aspect_ratio), frameon=False)

# Plotting
img = plt.imshow(
    grid_matrix,
    extent=(lon_min, lon_max, lat_min, lat_max),
    cmap='turbo',       # or 'RdYlGn_r' for Red-Green
    interpolation='nearest', # 'nearest' = Hard square edges (Pixelated grid)
    origin='upper'
)

plt.axis('off') # Remove axis ticks/border

# This saves just the heatmap content with transparent background
plt.savefig('grid_heatmap.png', bbox_inches='tight', pad_inches=0, dpi=300, transparent=True)
plt.close()

print("Grid heatmap saved as 'grid_heatmap.png'")


Generating grid: 100 x 133


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_regression.py:508: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)


Grid heatmap saved as 'grid_heatmap.png'


# Model for Individual Accessibility Index (IAI)

In [45]:
# 1. Load Data
df = pd.read_csv('IAI.csv')
# Modify the column to change the column
col_index = 2;

# Remove rows where Coordinates OR Score are missing (NaN)
df = df.dropna(subset=['geometry/coordinates/1', 'geometry/coordinates/0', df.columns[col_index]])

# 2. Prepare Coordinates
target_col = df.columns[col_index]
X = np.radians(df[['geometry/coordinates/1', 'geometry/coordinates/0']].values)
y = df[target_col].values

# 3. Model
radius_miles = 0.8
radius_radians = radius_miles / 3959.0

model = RadiusNeighborsRegressor(
    radius=radius_radians,
    weights='distance',
    metric='haversine',
    algorithm='ball_tree'
)

model.fit(X, y)


RadiusNeighborsRegressor(algorithm='ball_tree', metric='haversine',
                         radius=0.0002020712301086133, weights='distance')

In [40]:
def get_score(lat, lon):
    """
    Predicts score for a given lat/lon based on neighbors within 0.8 miles.
    """
    query_point = np.radians([[lat, lon]])

    try:
        score = model.predict(query_point)[0]
        return round(score, 4)
    except Exception:
        return None

# Example Usage: Prediction for a point in Seattle
print(f"Predicted Score: {get_score(47.61053462,-122.3408738)}")

Predicted Score: 0.8169


In [46]:
# 1. Define Grid Bounds & Resolution
lat_min, lat_max = df['geometry/coordinates/1'].min(), df['geometry/coordinates/1'].max()
lon_min, lon_max = df['geometry/coordinates/0'].min(), df['geometry/coordinates/0'].max()

# Resolution: Adjust '500' to change how "blocky" the grid is
# 500 means the image will be 500 squares wide
width_pixels = 100
aspect_ratio = (lat_max - lat_min) / (lon_max - lon_min)
height_pixels = int(width_pixels * aspect_ratio) # Keep map proportions correct

print(f"Generating grid: {width_pixels} x {height_pixels}")

# 2. Generate Grid Centers
lat_linspace = np.linspace(lat_min, lat_max, height_pixels)
lon_linspace = np.linspace(lon_min, lon_max, width_pixels)
grid_x_lon, grid_y_lat = np.meshgrid(lon_linspace, lat_linspace)

# Flatten to list of (Lat, Lon) points for batch prediction
# Note: grid_y_lat is Lat, grid_x_lon is Lon
query_points = np.column_stack((grid_y_lat.ravel(), grid_x_lon.ravel()))
query_points_rad = np.radians(query_points)

# 3. Batch Prediction with Zero-Filling
# We predict in chunks if memory is an issue, but for <1M points, one go is fine.
try:
    # RadiusNeighborsRegressor returns 0.0 by default for points with no neighbors
    # if you set it up correctly, or raises an error.
    # To be safe for "Transparency", we create a mask.

    # Get neighbors count first to identify "empty" cells (transparency mask)
    neighbor_graph = model.radius_neighbors(query_points_rad, return_distance=False)
    has_neighbors = np.array([len(n) > 0 for n in neighbor_graph])

    # Predict scores
    raw_scores = model.predict(query_points_rad)

    # Apply transparency logic: Set score to NaN where no neighbors exist
    # (Matplotlib treats NaN as transparent automatically)
    final_scores = np.where(has_neighbors, raw_scores, np.nan)

except Exception as e:
    print(f"Prediction logic warning: {e}")
    # Fallback: just predict and assume 0s
    final_scores = model.predict(query_points_rad)
    final_scores[final_scores == 0] = np.nan # Assume exactly 0.0 means "no data"

# 4. Reshape for Image
# Reshape to (Height, Width)
grid_matrix = final_scores.reshape(height_pixels, width_pixels)

# Flip Y-axis (Images start from top-left, Maps start from bottom-left)
grid_matrix = np.flipud(grid_matrix)

# 5. Save as PNG
plt.figure(figsize=(12, 12 * aspect_ratio), frameon=False)

# Plotting
img = plt.imshow(
    grid_matrix,
    extent=(lon_min, lon_max, lat_min, lat_max),
    cmap='turbo',       # or 'RdYlGn_r' for Red-Green
    interpolation='nearest', # 'nearest' = Hard square edges (Pixelated grid)
    origin='upper'
)

plt.axis('off') # Remove axis ticks/border

# This saves just the heatmap content with transparent background
if (col_index == 0):
    name = 'IAI_Wheel_chair_grid_heatmap.png'
elif (col_index == 1):
    name = 'IAI_Cane_grid_heatmap.png'
elif (col_index == 2):
    name = 'IAI_Pedestrian_grid_heatmap.png'
plt.savefig(name, bbox_inches='tight', pad_inches=0, dpi=300, transparent=True)
plt.close()

print("Grid heatmap saved as 'grid_heatmap.png'")


Generating grid: 100 x 133


/usr/local/lib/python3.12/dist-packages/sklearn/neighbors/_regression.py:508: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)


Grid heatmap saved as 'grid_heatmap.png'
